In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')
from sklearn.model_selection import train_test_split

In [2]:
%load_ext autoreload
%autoreload 2
import CCPA_lib as cp


In [3]:
sns.set_context('poster')
sns.set_style('white')

In [4]:
ppallete = 'RdYlGn'#'YlGn_d' #"BuGn_d"
porder = ['MED4',  'MIT9312','MIT0604', 'Natl2A', 'MIT9313']
ppallete = [ '#62A586', '#face6e','#f79934', '#7A9BC8', '#F47681', ]
# #ffd7a1
    
pedge = 'green'
apallete = "spring"
aorder = ['DE', 'DE1', '1A3', 'ATCC', 'BS11']
amarkers = ['X', 'P', 's', 'D', 'o']
aedge='orange'
epallete = ['#6cf0ba', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']
#sns.diverging_palette(355, 217, s=74, l=50, n=4) #'Paired' # 'BrBG'# "coolwarm"
eorder = ['e1','e3', 'e4', 'e5', 'e6']
#aedge='orange'
e2_color= '#f2f2f2'
eallorder= ['e1','e2', 'e3', 'e4', 'e5', 'e6']
eallpallete = ['#6cf0ba', '#f2f2f2', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']

efcmorder= ['e1','e2', 'e4', 'e5', ]
efcmpallete = ['#6cf0ba', '#f2f2f2',  '#eeb7cb', '#b9d2d8',]

espallete = ['#6cf0ba', '#D55282', '#538B9B']
#  604 #FCC479
#     #ffd7a1
#     #fcb14c
#     MED4 = #62A586
#     9313 #F47681
#     natl #7A9BC8

In [5]:
ppallete = 'RdYlGn'#'YlGn_d' #"BuGn_d"
porder = ['MED4',  'MIT9312','MIT0604', 'Natl2A', 'MIT9313']
ppallete = [ '#62A586', '#face6e','#f79934', '#7A9BC8', '#F47681', ]
psizes = [150,120,200,120,350]
pmarkers = ['o', 's', '^', 'D', '*']

# #ffd7a1
    
pedge = 'green'
apallete = "spring"
aorder = ['DE', 'DE1', '1A3', 'ATCC', 'BS11']
amarkers = ['P', 'X', 'h', '<','>']
apallete = ['#6dad36', '#c7e89f', '#f5c4e1', '#d5579d', '#8a0850']

aedge='orange'
epallete = ['#6cf0ba', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']
#sns.diverging_palette(355, 217, s=74, l=50, n=4) #'Paired' # 'BrBG'# "coolwarm"
eorder = ['e1','e3', 'e4', 'e5', 'e6']
#aedge='orange'

espallete = ['#6cf0ba', '#D55282', '#538B9B']

In [6]:
df = pd.read_pickle('CCPA.pkl.gz')
df.loc[df.PRO == 'C9B', 'PRO'] = 'MIT0604'

In [17]:
fcmdflist=list()
for i in os.listdir('fcmfiles'):
    if i.endswith('.csv'):
        d = pd.read_csv(os.path.join('fcmfiles', i))
        n = os.path.splitext(i)[0]
        _, experiment, day = n.split('_')
        d['experiment'] = experiment
        d['day'] = day.replace('Days','')
        fcmdflist.append(d)
fcmdf = pd.concat(fcmdflist)
fcmdf.loc[fcmdf.PRO == 'C9B', 'PRO'] = 'MIT0604'
fcmdf['day'] = pd.to_numeric(fcmdf['day'])

In [7]:
df.head()

,day,experiment,sample,FL,PRO,ALT,culture,FL_orig,logFL,cumsumFL,cumsumlogFL,zscoreFL,diffFL,difflogFL,diffday,rateFL,ratelogFL,experiment_sample
0,0.000000,e1,1A,0.133,MED4,1A3,Co_Culture,0.133,-2.017406,0.133,-2.017406,-0.588889,NaN,NaN,NaN,NaN,NaN,"e1, 1A"
1,0.715972,e1,1A,0.159,MED4,1A3,Co_Culture,0.159,-1.838851,0.292,-3.856257,-0.575917,0.026,0.178555,0.715972,0.036314,0.249388,"e1, 1A"
2,1.925694,e1,1A,0.155,MED4,1A3,Co_Culture,0.155,-1.864330,0.447,-5.720587,-0.577913,-0.004,-0.025479,1.209722,-0.003307,-0.021062,"e1, 1A"
3,2.831250,e1,1A,0.180,MED4,1A3,Co_Culture,0.180,-1.714798,0.627,-7.435386,-0.565440,0.025,0.149532,0.905556,0.027607,0.165127,"e1, 1A"
4,6.674306,e1,1A,0.632,MED4,1A3,Co_Culture,0.632,-0.458866,1.259,-7.894252,-0.339936,0.452,1.255933,3.843056,0.117615,0.326806,"e1, 1A"


In [8]:
def calculate_lag(d):
    minday_fl_threshold = 0.1
    r2_threshold = 0.9

    d = d.reset_index(drop=True)
    maxindex = d.FL.idxmax()
    maxday = d.loc[maxindex]['day']
    minday = d.loc[(d.day < maxday) & (d.FL <= minday_fl_threshold), 'day'].max()
    if minday is np.NaN:
        minday = d.day.min()
    
    minday = min(maxday-7,minday)
    
    reg, intercept, coef, r2 = cp.fit_regression(d.loc[(d.day > minday) & 
                                                       (d.day <= maxday)], 'day', 'logFL')
                        
    L = maxday - ((d.loc[maxindex]['logFL'] - d.loc[0]['logFL']) / coef)
    return pd.Series({
        'maxday' : maxday,
        'minday' : minday,
        'maxlogFL' : d.loc[maxindex]['logFL'],
        'logFL0' : d.loc[0]['logFL'],
        'log10FL0' : np.log10(d.loc[0]['FL']),
        'intercept': intercept,
        'coef': coef,
        'r2': r2,
        'L' : L,
    })

def get_prev_max_day_df(d, minmaxday_threshold):
    d = d.reset_index(drop=True)
    maxindex = d.FL.idxmax()
    maxday = d.loc[maxindex]['day']
    maxdaysdf = d.loc[(d.FL > d.FL.shift()) & (d.FL > d.FL.shift(-1)) & 
                    (d.day < maxday)].reset_index(drop=True)
    if maxdaysdf.shape[0] > 0:
        newmaxindex = maxdaysdf.FL.idxmax()
        newmaxday = maxdaysdf.loc[newmaxindex]['day']

        if ((newmaxday is not np.NaN) and #(newmaxday >= minmaxday_threshold) and 
            (d.loc[d.day == newmaxday, 'FL'].max() > (d.FL.max() / 2))):
            df_prevmax = d.loc[d.day < newmaxday]
            return df_prevmax
    return None

def get_no_outlier_df(d, minday_fl_threshold):
    d_no_outlier = d.loc[(d.FL > minday_fl_threshold) | 
                         (d.FL.shift() < minday_fl_threshold*2) |
                         (d.FL.shift(-1) < minday_fl_threshold*2)
                        ]
    return d_no_outlier

def get_without_max_df(d):
    d = d.reset_index(drop=True)
    maxindex = d.FL.idxmax()
    maxday = d.loc[maxindex]['day']
    d_withoutmaxday = d.loc[d.day < maxday]
    return d_withoutmaxday

def get_df_list(d, minday_fl_threshold, minmaxday_threshold):
    dnomax = get_without_max_df(d)
    dnomax2 = get_without_max_df(dnomax)
    
    df_list = [d, dnomax, dnomax2]
    df_list.extend([get_no_outlier_df(x, minday_fl_threshold) for x in df_list])
    return df_list

def calculate_lag_wrap(d):
    minday_fl_threshold = 0.07
    minmaxday_threshold = 4
    df_list = get_df_list(d, minday_fl_threshold, minmaxday_threshold)
    d2 = get_prev_max_day_df(d,minmaxday_threshold)
    if d2 is not None:
        df_list.extend(get_df_list(d2, minday_fl_threshold, minmaxday_threshold))
    reslist = [calculate_lag(x) for x in df_list]

    sorted_reslist = sorted(reslist, key=lambda x : x['r2'], reverse=True)
    return sorted_reslist[0]           

In [9]:
def computeL(maxday, maxlogcells, logcells0, coef):
    L = maxday - ((maxlogcells - logcells0) / coef)
    return L


In [10]:
def computeMaxDay(maxlogcells, logcells0, coef, L):
    maxday = ((maxlogcells - logcells0) / coef) + L
    return maxday


In [11]:
df = df.loc[~df.experiment.isin(['e7'])]

In [12]:
lagsdf = df.groupby(['experiment', 'sample', 'PRO', 'ALT', 'culture', 'experiment_sample']).apply(calculate_lag_wrap)\
        .reset_index()

In [13]:
lagsdf.head()

,experiment,sample,PRO,ALT,culture,experiment_sample,maxday,minday,maxlogFL,logFL0,log10FL0,intercept,coef,r2,L
0,e1,10A,MIT0604,DE,Co_Culture,"e1, 10A",14.960417,0.715972,1.726510,-2.813411,-1.221849,-2.590074,0.297113,0.991489,-0.319707
1,e1,10B,MIT0604,DE,Co_Culture,"e1, 10B",14.960417,0.715972,1.733424,-2.975930,-1.292430,-2.679265,0.304705,0.982196,-0.495052
2,e1,10C,MIT0604,DE,Co_Culture,"e1, 10C",14.960417,0.000000,1.741168,-2.718101,-1.180456,-2.626339,0.299733,0.989606,0.082945
3,e1,11A,MED4,DE1,Co_Culture,"e1, 11A",10.603472,0.000000,1.076026,-1.890475,-0.821023,-2.380434,0.326145,0.998481,1.507826
4,e1,11B,MED4,DE1,Co_Culture,"e1, 11B",13.694444,0.000000,1.810418,-1.698269,-0.737549,-2.258576,0.293950,0.998751,1.758092


In [14]:
lagsdf.columns

Index(['experiment', 'sample', 'PRO', 'ALT', 'culture', 'experiment_sample',
       'maxday', 'minday', 'maxlogFL', 'logFL0', 'log10FL0', 'intercept',
       'coef', 'r2', 'L'],
      dtype='object')

In [15]:
lagsdf = lagsdf.loc[~lagsdf.experiment.isin(['e7'])]

In [16]:
fcmdf.head()

NameError: name 'fcmdf' is not defined

In [ ]:
fcmdf.loc[(fcmdf.experiment == 'exp1') & (fcmdf.day == 60), 'e'] = 'e2'
fcmdf.loc[(fcmdf.experiment == 'exp1') & (fcmdf.day == 100), 'e'] = 'e4'
fcmdf.loc[(fcmdf.experiment == 'exp1') & (fcmdf.day == 140), 'e'] = 'e5'
fdf = fcmdf.dropna(subset=['e'])
fdf.loc[:,'experiment_sample'] = fdf['e'] + ', '+ fdf['sample']
fdf['initialcells'] = fdf['PRO.1']/20
fdf['initiallogcells'] = np.log(fdf['PRO.1']/20)
fdf['initiallog10cells'] = np.log10(fdf['PRO.1']/20)

fdf.head()

In [ ]:
fdf.columns

In [ ]:
lagsdf = pd.merge(lagsdf, fdf[['experiment_sample', 'initialcells', 'initiallogcells',
       'initiallog10cells']], on='experiment_sample', how='left')
lagsdf.loc[lagsdf.experiment == 'e1', 'initiallogcells'] = np.log(1e6)


In [ ]:
lagsdf.loc[lagsdf.experiment == 'e1', 'initialcells'] = 1e6
lagsdf.loc[lagsdf.experiment == 'e1', 'initiallog10cells'] = np.log10(1e6)


In [ ]:
lagsdf.loc[lagsdf['sample'] == '8C']

In [ ]:
lagsdf.groupby('experiment')['initiallogcells'].describe().T

In [ ]:
lagsdf['L_init'] = lagsdf.apply(lambda x: computeL(x['maxday'], x['maxlogcells'], x['initiallogcells'], x['coef']), axis=1)

In [ ]:
e1coef_df = lagsdf.loc[lagsdf.experiment == 'e1', ['sample', 'coef', 'L_init'] ]
e1coef_df.index = e1coef_df['sample']

lagsdf['e1 coef'] = lagsdf['sample'].map(e1coef_df['coef'])
lagsdf['e1 L_init'] = lagsdf['sample'].map(e1coef_df['L_init'])


In [ ]:
lagsdf['L_init_e1coef'] = lagsdf.apply(
    lambda x: computeL(x['maxday'], x['maxlogcells'], x['initiallogcells'], x['e1 coef']), axis=1)

In [ ]:
lagsdf['maxday_e1'] = lagsdf.apply(
    lambda x: computeMaxDay(x['maxlogcells'], x['initiallogcells'], x['e1 coef'], x['e1 L_init']), axis=1)

In [ ]:
le1df = lagsdf.loc[lagsdf.experiment == 'e1']

In [ ]:
le1df.columns

In [ ]:
(le1df.maxday - le1df.maxday_e1).value_counts()

In [ ]:
sns.relplot(data=le1df, x='maxday', y='maxday_e1')

In [ ]:
sns.set_context('talk')
sns.set_style('white')


In [ ]:
lagsdf.columns

In [ ]:
maxdf = df.groupby(['experiment', 'sample', 'PRO', 'ALT', 'culture', 'experiment_sample'])['cells'].max()\
        .reset_index()

In [ ]:
sns.catplot(data=maxdf, x='experiment', y='cells', kind='box', aspect=1.3,
           order=eallorder, palette=eallpallete)
#plt.yscale('log')
plt.title('maximum number of cells')

In [ ]:
sns.catplot(data=maxdf, x='experiment', y='cells', kind='box', 
           order=eallorder, palette=eallpallete,
            aspect=0.8, height=5,
            col='PRO', col_order=porder)
plt.suptitle('maximum number of cells', y=1.05)
    

In [ ]:
for i in ['maxday', 'maxlogcells', 'logcells0', 'log10cells0', 'intercept', 'coef', 'r2', 'L', 
          'L_init', 'e1 coef','L_init_e1coef', 'initialcells', 'initiallogcells',
       'initiallog10cells', 'maxday_e1']:
    sns.catplot(data=lagsdf, x='experiment', y=i, kind='box', aspect=1.3,
               order=eallorder, palette=eallpallete)
    plt.title(i)

In [ ]:
sns.set_style('white')
sns.set_context('poster')
for i in [#'maxday', 'maxlogcells', 'logcells0', 'log10cells0', 'intercept', 'coef', 'r2', 
    'L', 
          'L_init', #'e1 coef','L_init_e1coef', 'initialcells', 'initiallogcells',
       #'initiallog10cells', 'maxday_e1'
]:
    sns.catplot(data=lagsdf, x='experiment', y=i, kind='box', aspect=1.3, 
                order=efcmorder, palette=efcmpallete,
                facet_kws=dict(ylim=(-10,60))
                
               )
    plt.title(i)

In [ ]:
for i in [#'maxday', 'maxlogcells', 'logcells0', 'log10cells0', 'intercept', 'coef', 'r2',
    'L', 
          'L_init', #'e1 coef','L_init_e1coef', 'initialcells', 'initiallogcells',
       #'initiallog10cells', 'maxday_e1'
]:
    sns.catplot(data=lagsdf, x='experiment', y=i, kind='box', aspect=0.8, height=5,
                order=efcmorder,palette=efcmpallete,
                col='PRO', col_order=porder,
                facet_kws=dict(ylim=(-10,60))
               )
    plt.suptitle(i, y=1.05)

In [ ]:
for i in ['maxday', 'maxlogcells', 'logcells0', 'log10cells0', 'intercept', 'coef', 'r2', 'L', 
          'L_init', 'e1 coef','L_init_e1coef', 'initialcells', 'initiallogcells',
       'initiallog10cells', 'maxday_e1']:
    sns.catplot(data=lagsdf, x='experiment', y=i, kind='box', aspect=0.8, height=5,
                order=eallorder,palette=eallpallete,
                
                col='PRO', col_order=porder)
    plt.suptitle(i, y=1.05)

In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='log10cells0', y='initiallog10cells', hue='PRO', 
                height=4, aspect=1,
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                facet_kws=dict(ylim=(1,8), xlim=(6,8), legend_out=True)
            )
g.map(sns.lineplot, x=lagsdf['log10cells0'], y=lagsdf['log10cells0'], linestyle='dashed' )
g.set_axis_labels('log10(cells) (fit from FL)', 'log10(cells) - FCM')
plt.legend(bbox_to_anchor=(1.6,1), frameon=False)

In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='maxday', y='maxday_e1', hue='PRO', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                facet_kws=dict(ylim=(0,80), xlim=(0,80))
            )
g.map(sns.lineplot, x=lagsdf['maxday'], y=lagsdf['maxday'], linestyle='dashed' )
g.set_axis_labels('actual max day', 'computed max day')


In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
sns.set_context('talk')
sns.set_style('white')
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='e1 coef', y='coef', hue='PRO', 
            col='experiment', #col_wrap=2,
                col_order=['e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                facet_kws=dict(ylim=(0,0.6), xlim=(0,0.6)),
                legend=False
                
            )
g.map(sns.lineplot, x=lagsdf['e1 coef'], y=lagsdf['e1 coef'], linestyle='dashed' )
g.set_axis_labels('experiment 1 growth rate', 'growth rate')
#plt.legend(False)

In [ ]:
lagsdf.columns

In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='L_init_e1coef', y='L_init', hue='PRO', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                #facet_kws=dict(ylim=(0,0.6), xlim=(0,0.6))
            )
g.map(sns.lineplot, x=lagsdf['L_init_e1coef'], y=lagsdf['L_init_e1coef'], linestyle='dashed' )
g.set_axis_labels('Lag based on experiment 1 rate (days)', 'Lag based on FCM (days)')


In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='L_init_e1coef', y='L', hue='PRO', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                #facet_kws=dict(ylim=(0,0.6), xlim=(0,0.6))
            )
g.map(sns.lineplot, x=lagsdf['L_init_e1coef'], y=lagsdf['L_init_e1coef'], linestyle='dashed' )
g.set_axis_labels('Lag based on experiment 1 rate (days)', 'Lag (days)')


In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='L', y='L_init', hue='PRO', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, hue_order=porder,
                style='PRO', style_order=porder, markers=pmarkers,
                size='PRO', size_order=porder, sizes=psizes,
                #facet_kws=dict(ylim=(0,0.6), xlim=(0,0.6))
            )
g.map(sns.lineplot, x=lagsdf['L_init_e1coef'], y=lagsdf['L_init_e1coef'], linestyle='dashed' )
g.set_axis_labels('Lag (FL) (days)', 'Lag (FCM) (days)')


In [ ]:
#for e in ['e1', 'e2', 'e4', 'e5']: 
g = sns.relplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='maxday', y='maxday_e1', hue='ALT', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=apallete + ['black'], hue_order=aorder + ['Ax_Pro'],
                s=100,
                style='ALT', style_order=aorder+ ['Ax_Pro'], markers=amarkers + ['D'],
                facet_kws=dict(ylim=(0,80), xlim=(0,80))
            )
g.map(sns.lineplot, x=lagsdf['maxday'], y=lagsdf['maxday'], linestyle='dashed' )
g.set_axis_labels('actual max day', 'computed max day')


In [ ]:
lagsdf['max day diff (actual - computed)'] = lagsdf['maxday'] - lagsdf['maxday_e1']
g = sns.catplot(data=lagsdf.loc[lagsdf.culture.isin(['Co_Culture', 'Axenic'])], 
                                x='PRO', y='max day diff (actual - computed)', #hue='PRO', 
            col='experiment', col_wrap=2, col_order=['e1', 'e2', 'e4', 'e5'], 
                palette=ppallete, order=porder,
                aspect=1.2,
                #style='PRO', style_order=porder, markers=pmarkers,
                #size='PRO', size_order=porder, sizes=psizes,
                #facet_kws=dict(ylim=(0,80))
                kind='box',
            )
def plot_axhline0(**kwargs):
    plt.axhline(0,**kwargs)
g.map(plot_axhline0, ls=':')


In [ ]:
sns.set_context('talk')
sns.set_style('white')
for e in ['e1','e2','e4','e5']:
    g = sns.relplot(data=lagsdf.loc[lagsdf.experiment.isin([e]) & lagsdf.PRO.isin(porder)], 
            x='L', y='L_init',  hue='PRO', s=50, #legend=False,
                    hue_order=porder, palette=ppallete,
                    style='PRO', style_order=porder, markers=pmarkers,
                    size='PRO', size_order=porder, sizes=psizes,
                   )
    sns.lineplot(x=lagsdf.loc[lagsdf.experiment.isin([e]),'L'], 
                 y=lagsdf.loc[lagsdf.experiment.isin([e]),'L'], color='black', ax=g.ax, legend=False )
    g.set_axis_labels('L (days)', 'L based on FCM (days)')
    plt.suptitle(f'Lag vs Lag based on initial PRO cells - {e}', y=1.05)

In [ ]:
lagsdf.columns

In [ ]:
sns.set_context('talk')
sns.set_style('white')
for e in ['e1','e2','e4','e5']:
    g = sns.relplot(data=lagsdf.loc[lagsdf.experiment.isin([e]) & lagsdf.PRO.isin(porder)], 
            y='logcells0', x='initiallogcells',  hue='PRO', s=50, #legend=False,
                    hue_order=porder, palette=ppallete,
                    style='PRO', style_order=porder, markers=pmarkers,
                    size='PRO', size_order=porder, sizes=psizes,
                   )
    sns.lineplot(x=lagsdf.loc[lagsdf.experiment.isin([e]),'initiallogcells'], 
                 y=lagsdf.loc[lagsdf.experiment.isin([e]),'initiallogcells'], color='black', ax=g.ax, legend=False )
    g.set_axis_labels('log cells (FCM)', 'log cells (from FL)', )
    plt.suptitle(f'cells from FL vs FCM - {e}', y=1.05)
    plt.ticklabel_format(useOffset=False)

In [ ]:
worse_l = lagsdf.nsmallest(10, 'L')['experiment_sample']

In [ ]:
lagsdf.nsmallest(10, 'r2')

In [ ]:
def plot_growth(es):
    sns.set(rc={'figure.figsize':(10,5)})
    maxday = lagsdf.loc[lagsdf.experiment_sample == es, 'maxday'].unique()[0]
    minday = lagsdf.loc[lagsdf.experiment_sample == es, 'minday'].unique()[0]
    r2 = lagsdf.loc[lagsdf.experiment_sample == es, 'r2'].unique()[0]
    L = lagsdf.loc[lagsdf.experiment_sample == es, 'L'].unique()[0]
    
    d = df.loc[df.experiment_sample == es].reset_index()
    d['fitted_growth'] = d.day*lagsdf.loc[lagsdf.experiment_sample == es, 'coef'].unique() + lagsdf.loc[lagsdf.experiment_sample == es, 'intercept'].unique()
    d.loc[d.day > maxday, 'fitted_growth'] = np.NaN
    d.loc[d.day < minday, 'fitted_growth'] = np.NaN
    sns.lineplot(data=d,
                 x='day', y='logcells',  style=True, markers=True,label='logcells')
    sns.lineplot(data=d,
        x='day', y='fitted_growth',  style=True, markers=True,label='fit')
    plt.title(f'{es}: L={L:.2f}, r2={r2:.2f}')
    plt.show()


In [ ]:
for es in worse_l:
    plot_growth(es)

In [ ]:
worse_r2 = lagsdf.nsmallest(10, 'r2')['experiment_sample']
for es in worse_r2:
    plot_growth(es)

In [ ]:
sns.relplot(data=lagsdf, x='maxday', y='L', hue='experiment', size='r2')

In [ ]:
lagsdf.loc[lagsdf.maxday>60]

In [ ]:
def plot_growth(es):
    maxday = lagsdf.loc[lagsdf.experiment_sample == es, 'maxday'].unique()[0]
    minday = lagsdf.loc[lagsdf.experiment_sample == es, 'minday'].unique()[0]
    r2 = lagsdf.loc[lagsdf.experiment_sample == es, 'r2'].unique()[0]
    L = lagsdf.loc[lagsdf.experiment_sample == es, 'L'].unique()[0]
    
    d = df.loc[df.experiment_sample == es].reset_index()
    d['fitted_growth'] = d.day*lagsdf.loc[lagsdf.experiment_sample == es, 'coef'].unique() + lagsdf.loc[lagsdf.experiment_sample == es, 'intercept'].unique()
    d.loc[d.day > maxday, 'fitted_growth'] = np.NaN
    d.loc[d.day < minday, 'fitted_growth'] = np.NaN
    sns.lineplot(data=d,
                 x='day', y='logcells',  style=True, markers=True,label='logcells')
    sns.lineplot(data=d,
        x='day', y='fitted_growth',  style=True, markers=True,label='fit')
    plt.title(f'{es}: L={L:.2f}, r2={r2:.2f}')
    plt.show()


In [ ]:
def add_fit(d):
    es = d.name
    lagsdf_row = lagsdf.loc[lagsdf.experiment_sample == es].iloc[0]
    maxday = lagsdf_row['maxday']
    minday = lagsdf_row['minday']
    coef = lagsdf_row['coef']
    intercept = lagsdf_row['intercept']
    
    d['fitted_growth'] = d.day*coef + intercept
    d.loc[d.day > maxday, 'fitted_growth'] = np.NaN
    d.loc[d.day < minday, 'fitted_growth'] = np.NaN
    return d
    

df_fit = df.reset_index(drop=True).groupby(['experiment_sample']).apply(add_fit)


In [ ]:
df_fit.columns

In [ ]:
df_fit_m = df_fit.melt(id_vars=['day', 'experiment', 'sample', 'PRO', 'ALT', 'culture', 'experiment_sample'],
            value_vars=['logcells', 'fitted_growth'], value_name='ln(cell/ml)', var_name='var' )

In [ ]:

g = sns.relplot(data=df_fit_m.loc[df_fit_m['sample'].isin(['11C']) &
                               df_fit_m.experiment.isin(['e1','e2','e4','e5'])],
             x='day', y='ln(cell/ml)', hue='var',
                style=True, markers=True, col='experiment', col_wrap=2, kind='line')



In [ ]:
with pd.option_context("display.precision", 2,):
    display(lagsdf.loc[lagsdf['sample'].isin(['11C'])&
           lagsdf.experiment.isin(['e1','e2','e4','e5']),]
                       
#           #['experiment', 
#         'log10cells0', 'initiallog10cells',
#        'coef', 'r2', 'L', 
#        'L_init', 'L_init_e1coef', 
#       'maxday', 'maxday_e1', ]]
           )

In [ ]:
### es = 'e5, 11C'
69 , (16.04 - 3.33) / 0.11, 69  - (16.04 - 14) / 0.11, 
#L = maxday - ((maxlogcells - logcells0) / coef)


In [ ]:
lagsdf.nsmallest(10, 'initiallogcells')

In [ ]:
lagsdf.nsmallest(10, 'L_init').T

In [ ]:
sns.set(rc={'figure.figsize':(15,8)})
es = 'e3, 8B'
d = df.loc[df.experiment_sample == es].reset_index()
d['fitted_growth'] = d.day*lagsdf.loc[lagsdf.experiment_sample == es, 'coef'].unique() + lagsdf.loc[lagsdf.experiment_sample == es, 'intercept'].unique()
sns.lineplot(data=d,
             x='day', y='logcells',  style=True, markers=True,label='logFL')
sns.lineplot(data=d,
    x='day', y='fitted_growth',  style=True, markers=True,label='fit')

#sns.lineplot(data=d, x='day', y='first_deriviative',  style=True, markers=True,label='1')
#sns.lineplot(data=d, x='day', y='second_deriviative',  style=True, markers=True,label='2')

In [ ]:
sns.relplot(data=df.loc[df.experiment == 'e2'], x='day', y='FL', 
            col='sample', hue='PRO', col_wrap=4, kind='line')

In [ ]:
sns.catplot(data=lagsdf.loc[lagsdf.experiment == 'e1'],
            x='culture', y='coef', kind='box',
           )

In [ ]:
sns.catplot(data=lagsdf.loc[lagsdf.experiment == 'e1'],
            hue='culture', y='coef', kind='box', x='PRO', aspect=3,
           )